In [52]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
# 数据加载
Train_data = pd.read_csv('./train_data.csv', index_col=0)
Test_data = pd.read_csv('./test_data.csv', index_col=0)
Test_data.drop('Attrition', axis=1, inplace=True)
x_train, x_test, y_train, y_test = train_test_split(Train_data.drop('Attrition', axis=1), Train_data['Attrition'], test_size=0.2)

In [53]:
Train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176 entries, 0 to 1175
Data columns (total 31 columns):
Age                         1176 non-null int64
Attrition                   1176 non-null int64
BusinessTravel              1176 non-null int64
DailyRate                   1176 non-null int64
Department                  1176 non-null int64
DistanceFromHome            1176 non-null int64
Education                   1176 non-null int64
EmployeeCount               1176 non-null int64
EnvironmentSatisfaction     1176 non-null int64
Gender                      1176 non-null int64
HourlyRate                  1176 non-null int64
JobInvolvement              1176 non-null int64
JobLevel                    1176 non-null int64
JobRole                     1176 non-null int64
JobSatisfaction             1176 non-null int64
MaritalStatus               1176 non-null int64
MonthlyIncome               1176 non-null int64
MonthlyRate                 1176 non-null int64
NumCompaniesWorked          1

In [54]:
Test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294 entries, 1176 to 1469
Data columns (total 30 columns):
Age                         294 non-null int64
BusinessTravel              294 non-null int64
DailyRate                   294 non-null int64
Department                  294 non-null int64
DistanceFromHome            294 non-null int64
Education                   294 non-null int64
EmployeeCount               294 non-null int64
EnvironmentSatisfaction     294 non-null int64
Gender                      294 non-null int64
HourlyRate                  294 non-null int64
JobInvolvement              294 non-null int64
JobLevel                    294 non-null int64
JobRole                     294 non-null int64
JobSatisfaction             294 non-null int64
MaritalStatus               294 non-null int64
MonthlyIncome               294 non-null int64
MonthlyRate                 294 non-null int64
NumCompaniesWorked          294 non-null int64
OverTime                    294 non-null int6

In [55]:
attr=['BusinessTravel','Department','Education','JobRole','MaritalStatus','OverTime']

In [56]:
param = {'boosting_type':'gbdt',
                         'objective' : 'binary', #
                         #'metric' : 'binary_logloss',
                         'metric' : 'auc',
#                          'metric' : 'self_metric',
                         'learning_rate' : 0.01,
                         'max_depth' : 15,
                         'feature_fraction':0.8,
                         'bagging_fraction': 0.9,
                         'bagging_freq': 8,
                         'lambda_l1': 0.6,
                         'lambda_l2': 0,
#                          'scale_pos_weight':k,
#                         'is_unbalance':True
        }

In [58]:
X_train, X_valid, y_train, y_valid = train_test_split(Train_data.drop('Attrition',axis=1), Train_data['Attrition'], test_size=0.2, random_state=42)
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)


model = lgb.train(param,train_data,valid_sets=[train_data,valid_data],num_boost_round = 10000 ,
                  early_stopping_rounds=200,verbose_eval=25,categorical_feature=attr)
predict=model.predict(Test_data)

/Users/wangjian/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['BusinessTravel', 'Department', 'Education', 'JobRole', 'MaritalStatus', 'OverTime']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds
[25]	training's auc: 0.941656	valid_1's auc: 0.778675
[50]	training's auc: 0.949277	valid_1's auc: 0.791444
[75]	training's auc: 0.956116	valid_1's auc: 0.795482
[100]	training's auc: 0.964083	valid_1's auc: 0.803121
[125]	training's auc: 0.970478	valid_1's auc: 0.808251
[150]	training's auc: 0.977977	valid_1's auc: 0.81087
[175]	training's auc: 0.983186	valid_1's auc: 0.80956
[200]	training's auc: 0.987525	valid_1's auc: 0.811197
[225]	training's auc: 0.990506	valid_1's auc: 0.811852
[250]	training's auc: 0.992797	valid_1's auc: 0.812507
[275]	training's auc: 0.995114	valid_1's auc: 0.816872
[300]	training's auc: 0.996353	valid_1's auc: 0.81709
[325]	training's auc: 0.997494	valid_1's auc: 0.818509
[350]	training's auc: 0.998204	valid_1's auc: 0.820692
[375]	training's auc: 0.998787	valid_1's auc: 0.822438
[400]	training's auc: 0.999281	valid_1's auc: 0.825603
[425]	training's auc: 0.999488	valid_1's auc: 0.825385
[450]	tra

In [72]:
# Test_data['Attrition']=predict
# Test_data['Attrition']=Test_data['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test['Attrition']=predict
temp = pd.read_csv('./test.csv')['user_id']
# print(temp)
submit = pd.DataFrame(temp)
submit['Attrition'] = predict
submit['Attrition'] = submit['Attrition'].map(lambda x:1 if x>=0.5 else 0)
print(submit)
submit.to_csv('submit_lgb.csv', index=None)

     user_id  Attrition
0        442          0
1       1091          0
2        981          0
3        785          0
4       1332          1
..       ...        ...
289     1439          0
290      481          0
291      124          1
292      198          0
293     1229          0

[294 rows x 2 columns]
